In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

from transformers import TextDataset
from torch.utils.data import DataLoader, Dataset
import torch
from torch.nn.functional import cross_entropy
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast, AdamW

from datasets import load_dataset

In [ ]:
# для отладки проблем с cuda
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
# Первая часть данных, датасет с Rucode
train = pd.read_csv('/kaggle/input/spell-check-dataset-rucode/train.csv', usecols=['corrupted_text', 'correct_text'])

train = train.rename({'corrupted_text':'incorrect', 'correct_text':'correct'}, axis=1)
train = train.sample(26000).reset_index(drop=True)

In [4]:
train_dataset = load_dataset("ai-forever/spellcheck_benchmark", "RUSpellRU", split='train[:]')
eval_dataset = load_dataset("ai-forever/spellcheck_benchmark", "RUSpellRU", split='test[:]')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset russian_spellcheck_benchmark downloaded and prepared to /root/.cache/huggingface/datasets/ai-forever___russian_spellcheck_benchmark/RUSpellRU/0.0.1/87bfa2950c7b82ec565b4da426533874af24d25436ad08dba065a45895ad3945. Subsequent calls will reuse this data.


In [5]:
first_df = pd.DataFrame({'incorrect':train_dataset['source'], 'correct': train_dataset['correction']})
second_df = pd.DataFrame({'incorrect':eval_dataset['source'], 'correct': eval_dataset['correction']})

full_df = pd.concat([first_df, second_df, train])
full_df['incorrect'] = 'Spell correct: ' + full_df['incorrect']

In [23]:
first_df.sample(5)

,incorrect,correct
534,"Как я понял, это изначально сделано.",Как я понял это изначально сделано
823,"Я вам покажу, как будить Колдуна!",Я вам покажу как будить Колдуна
1198,"Я думала, кстате, что даже у маленбких утят пе...",Я думала кстати что даже у маленьких утят перь...
578,"А в 8.30 я пошагала на пары, а вечером дописыв...",А в 8.30 я пошагала на пары а вечером дописыва...
1285,"Мужык был в транче и тихо повторял - "" меня в ...",Мужик был в трансе и тихо повторял меня в горо...


In [45]:
MODEL_NAME = 'google/mt5-small'
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
class SpellCheckDataset(Dataset):
    def __init__(self, df:pd.DataFrame):
        self.inputs = df.iloc[:, 0].values
        self.outputs = df.iloc[:, 1].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        return tokenizer(input_text, padding=False, truncation=True), tokenizer(output_text, padding=False, truncation=True)
    
def collate_fn(batch):
    input_texts = [item[0] for item in batch]
    output_texts = [item[1] for item in batch]

    input_texts = tokenizer.pad(input_texts, return_tensors="pt", padding='longest')
    output_texts = tokenizer.pad(output_texts, return_tensors="pt", padding='longest')

    return input_texts, output_texts

def bucketed_data_loader(dataframe: pd.DataFrame, batch_size:int):
    dataframe = dataframe.copy()
    dataframe['length'] = dataframe.iloc[:, 0].progress_apply(lambda x: len(tokenizer.encode(x, truncation=True)))
    dataframe.sort_values(by='length', inplace=True)

    dataset = SpellCheckDataset(dataframe.drop(columns=['length']))

    sampler = torch.utils.data.BatchSampler(
        torch.utils.data.SequentialSampler(dataset),
        batch_size=batch_size,
        drop_last=False
    )

    return DataLoader(dataset, batch_sampler=sampler, collate_fn=collate_fn)

batch_size = 8
loader = bucketed_data_loader(full_df, batch_size)

100%|██████████| 30008/30008 [00:03<00:00, 9296.67it/s] 


In [8]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
num_epochs = 40
model.train()

for epoch in range(num_epochs):
    for batch_idx, batch in tqdm(enumerate(loader)):
        optimizer.zero_grad()

        input_ids = batch[0]["input_ids"].to(device)
        attention_mask = batch[0]["attention_mask"].to(device)
        labels = batch[1]["input_ids"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        if batch_idx % 240 == 9:
            print(f"Epoch: {epoch + 1}, Batch: {batch_idx}/{len(loader)}, Loss: {loss.item()}")

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
11it [00:05,  5.48it/s]

Epoch: 1, Batch: 9/3751, Loss: 1.9971446990966797


251it [00:41,  6.70it/s]

Epoch: 1, Batch: 249/3751, Loss: 0.43817901611328125


491it [01:18,  6.85it/s]

...

Epoch: 6, Batch: 2409/3751, Loss: 0.1172742173075676


2651it [06:30,  6.84it/s]

Epoch: 6, Batch: 2649/3751, Loss: 0.03495532646775246


2697it [06:37,  6.78it/s]


KeyboardInterrupt: 

# model inference

In [10]:
def restore_punctuation(original_text, good_text):
    res = []
    punctuation = '''!@#$%^&*(){}[]|._`/?:;"'\,~'''
    for orig_word, word in zip(original_text.split(), good_text.split()):
        if orig_word[-1] in punctuation:
            word = word + orig_word[-1]
        if orig_word[0] in punctuation:
            word = orig_word[0] + word

        if (orig_word == '-' or orig_word == '—') and word == 'я':
            word = orig_word

        if orig_word.istitle():
            word = word.capitalize()

        res.append(word)

    restored_text = ' '.join(res)
    for punct in punctuation:
        restored_text = restored_text.replace(' ' + punct, punct)

    return restored_text

In [42]:
prefix = 'Spell correct: '
input_text = 'Молако! Надя решила прикинутся мервтой'
input_ids = tokenizer.encode(prefix+input_text, return_tensors='pt')

input_ids = input_ids.to(device)

with torch.no_grad():
    outputs = model.generate(input_ids)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [43]:
from IPython.display import display, HTML

# для подкрашивания измененных слов в получившемся тексте
def highlight_corrections(original_text, corrected_text):
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    def create_highlighted_html(words, corrections):
        highlighted_html = ""
        for word, is_corrected in zip(words, corrections):
            if is_corrected:
                highlighted_html += f"<mark>{word}</mark> "
            else:
                highlighted_html += word + " "
        return highlighted_html

    corrections = [ow != cw for ow, cw in zip(original_words, corrected_words)]
    highlighted_original = create_highlighted_html(original_words, corrections)
    highlighted_corrected = create_highlighted_html(corrected_words, corrections)

    display(HTML(f"Оригинальный текст: {highlighted_original}<br><br>Исправленный текст: {highlighted_corrected}"))

In [44]:
highlight_corrections(input_text, restore_punctuation(input_text, output_text))